# Website Recovery Procedure

 > You will review the steps within the documented procedure for remediation of website issues.

# Table of Contents
 - [Prerequisites](#Prerequisites)
   - [Install Modules](#install-modules)
   - [Testing Your Site](#testing-your-site)
 - [AWS Discovery](#aws-discovery)
   - [Review Route 53 Configuration](#review-route-53-configuration)
   - [Review Elastic Load Balancer (ELB) Configuration](#review-elastic-load-balancer-(ELB)-configuration)
   - [Review Elastic Compute Cloud (EC2) Configuration](#review-elastic-compute-cloud-(EC2)-configuration)
 - [Remediate](#remediate)
 - [Appendix](#appendix)

In [3]:
#!mermaid
flowchart TB
    subgraph one [Prerequisites]
        direction LR
        s1(Install Modules) --> s2(Testing Your Site)
    end
    subgraph two [AWS Discovery]
        direction LR
        s5(Review Route 53 Config) --> s6(Review ELB Config)
        s6 --> s7(Review EC2 Config)
    end
    one ==> two
    two ==> three(Remediate)

# Prerequisites

## Install Modules

You will use PowerShell to work with the [AWS Tools for PowerShell](https://aws.amazon.com/powershell/) and to do this you will require the use of a few modules.

Begin by:
 - Identifying the core required modules using the [array sub-expression operator](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_arrays#the-array-sub-expression-operator)

In [ ]:
$modules=@(
    "AWS.Tools.Common"
    "AWS.Tools.Installer"
)

* The AWS Tools for PowerShell will also use a separate installation process for the service specific modules

In [ ]:
$awsModules=@(
    "AWS.Tools.EC2"
    "AWS.Tools.ElasticLoadBalancingV2"
    "AWS.Tools.Route53"
)

 - Identifying currently available AWS Tools for PowerShell modules

> ---
> ✒️ **_NOTE:_** Below you will use the `?` alias for Where-Object and the `$_` alias for [`$PSItem`](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_psitem)
>
> ---

In [ ]:
$available = Get-Module -ListAvailable|?{$_.Name -like "AWS.Tools.*"}

 - Then if a necessary module is not available you will install it

In [ ]:
foreach($module in $modules)
{
    if($module -notin $available.Name)
    {
        Install-Module $module -Force
    }
}

foreach($module in $awsModules)
{
    if($module -notin $available.Name)
    {
        Install-AWSToolsModule $module -Force
    }
}

## Testing Your Site

Test to see if the public URL is reachable

In [ ]:
Invoke-RestMethod https://demo.soule.aws.sentinel.com

# AWS Discovery

> ---
> ℹ️ **_Information:_** Your JupyterLab instance runs as a Fargate task and inherits an Identity Access Management (IAM) Role, so you do not need to explicitly authenticate.
>
> ---

## Review Route 53 Configuration

Now that you have the necessary modules, you can begin reviewing the existing environment. You will begin identifying resource dependencies from the outside in as you know the website's public URL.

 - Begin by looking up the Route 53 Zone

In [ ]:
$id = (Get-R53HostedZonesByName|?{$_.Name -like "soule.*"}).Id
$id

In [ ]:
$sets = (Get-R53ResourceRecordSet $id).ResourceRecordSets
$sets|select -First 1

In [ ]:
$records = ($sets|?{$_.Name -like "demo.*"}).ResourceRecords.Value
$records

### Testing Using Record Value

In [ ]:
$records|%{irm "https://$_" -SkipCertificateCheck}

## Review Elastic Load Balancer (ELB) Configuration

In [ ]:
$lb = Get-ELB2LoadBalancer|?{$_.DnsName -in $records}
$lb

In [ ]:
$group = Get-ELB2TargetGroup -LoadBalancerArn $lb.LoadBalancerArn
$group

In [ ]:
$targetHealth = Get-ELB2TargetHealth $group.TargetGroupArn
$targetHealth

In [ ]:
$targetHealth|select @{n="target";e={$_.Target.Id}},@{n="health";e={$_.TargetHealth.State}}

In [ ]:
$instances = (Get-EC2Instance|?{$_.Instances.InstanceId -in $targetHealth.Target.Id}).Instances
$instances

### Testing Using Targets

In [ ]:
$instances|%{irm "$($group.HealthCheckProtocol)://$($_.PrivateIpAddress):$($group.Port)$($group.healthCheckPath)"}

## Review Elastic Compute Cloud (EC2) Configuration

In [ ]:
$instanceState = Get-EC2InstanceStatus -InstanceIds $instances -IncludeAllInstances $true|select InstanceId,@{n="InstanceState";e={$_.InstanceState.Name}},@{n="InstanceStatus";e={$_.Status.Details}}
$instanceState

### Identify Stopped Targets

In [ ]:
$stoppedInstances = $instanceState|?{$_.InstanceState -notin @("running","pending")}
$stoppedInstances

# Remediate

In [ ]:
$stoppedInstances|%{Start-EC2Instance $_.InstanceId}

In [ ]:
$instanceState = Get-EC2InstanceStatus -InstanceIds $instances -IncludeAllInstances $true|select InstanceId,@{n="InstanceState";e={$_.InstanceState.Name}},@{n="InstanceStatus";e={$_.Status.Details}}
$instanceState

In [ ]:
$instances|%{irm "$($group.HealthCheckProtocol)://$($_.PrivateIpAddress):$($group.Port)$($group.healthCheckPath)"}

In [ ]:
$records|%{irm "https://$_" -SkipCertificateCheck}

In [ ]:
irm https://demo.soule.aws.sentinel.com

# Appendix

## Identifying API Operation to Cmdlet Mapping

In [ ]:
$apiOperations = @(
    "listHostedZonesByName",
    "listResourceRecordSets",
    "describeLoadBalancers",
    "describeTargetGroups",
    "describeTargetHealth",
    "describeInstances",
    "describeInstanceStatus"
)
foreach($operation in $apiOperations)
{
    Get-AWSCmdletName -ApiOperation $operation
}